 # Week 6: Midterm Project
 File: DSC540_Paulovici_Midterm.py (.ipynb)<br>
 Name: Kevin Paulovici<br>
 Date: 1/19/2020<br>
 Course: DSC 540 Data Preparation (2203-1)<br>
 Assignment: Midterm Project

 # Tasks
 1) Replace headers (Data Wrangling with Python pg. 154 – 163) <br>
 2) Format Data to a Readable Format (Data Wrangling with Python pg. 164 – 168) <br>
 3) Identify outliers and bad data (Data Wrangling with Python pg. 169 – 174) <br>
 4) Find Duplicates (Data Wrangling with Python pg. 175 – 178) <br>
 5) Conduct Fuzzy Matching (if you don’t have an obvious example to do this with in your data, create categories and use Fuzzy Matching to lump data together) (Data Wrangling with Python pg. 179 – 188) <br>
 Summary - A 250-word paper summarizing your steps and any challenges you ran into during the project. You should also outline any decisions you had to make while transforming the data (for example, if you decided to remove duplicates, how did you arrive at that decision?).

 ## Database Info
 Dataset: pokemon.csv retrieved from https://www.kaggle.com/rounakbanik/pokemon <br>
 Note: The dataset contains 802 rows, this was extended to more than the required 1000. This was also done for #4 of the assignment.

In [1]:
from csv import DictReader
import pprint


 ## Part 1 - Replace headers

In [2]:
# open the dataset wiith DictReader
poke_data = DictReader(open('pokemon_data.csv', encoding='ISO-8859-1'))

# create list of all rows
poke_rows = [d for d in poke_data]

pprint.pprint(poke_rows[0])


OrderedDict([('abilities', "['Overgrow', 'Chlorophyll']"),
             ('against_bug', '1'),
             ('against_dark', '1'),
             ('against_dragon', '1'),
             ('against_electric', '0.5'),
             ('against_fairy', '0.5'),
             ('against_fight', '0.5'),
             ('against_fire', '2'),
             ('against_flying', '2'),
             ('against_ghost', '1'),
             ('against_grass', '0.25'),
             ('against_ground', '1'),
             ('against_ice', '2'),
             ('against_normal', '1'),
             ('against_poison', '1'),
             ('against_psychic', '2'),
             ('against_rock', '1'),
             ('against_steel', '1'),
             ('against_water', '0.5'),
             ('attack', '49'),
             ('base_egg_steps', '5120'),
             ('base_happiness', '70'),
             ('base_total', '318'),
             ('capture_rate', '45'),
             ('classfication', 'Seed PokÃ©mon'),
             ('defense', '49

In [3]:
""" After looking at the headers, we can replace the "against_" col names with something
else, say "facing_". 
So, we need to make a dict of headers to replace. 
1) Find the the write headers
2) Add as a key to dict, replacement in value 
"""
new_headers = {} 

for data_dict in poke_rows:
    for dkey, dval in data_dict.items():
        # determine if it is a header we want to replace
        if "against_" in dkey:
            new_headers[dkey] = dkey.replace("against_", "facing_")


In [4]:
""" Search through the dicition, when headers are found that need to be replace. 
Otherwise, leave data as is
"""
new_poke_rows = []

for data_dict in poke_rows:
    new_row = {}
    for dkey, dval in data_dict.items():
        if dkey in new_headers.keys():
            new_row[new_headers[dkey]] = dval
        else:
            new_row[dkey] = dval
    new_poke_rows.append(new_row)

pprint.pprint(new_poke_rows[0])


{'abilities': "['Overgrow', 'Chlorophyll']",
 'attack': '49',
 'base_egg_steps': '5120',
 'base_happiness': '70',
 'base_total': '318',
 'capture_rate': '45',
 'classfication': 'Seed PokÃ©mon',
 'defense': '49',
 'experience_growth': '1059860',
 'facing_bug': '1',
 'facing_dark': '1',
 'facing_dragon': '1',
 'facing_electric': '0.5',
 'facing_fairy': '0.5',
 'facing_fight': '0.5',
 'facing_fire': '2',
 'facing_flying': '2',
 'facing_ghost': '1',
 'facing_grass': '0.25',
 'facing_ground': '1',
 'facing_ice': '2',
 'facing_normal': '1',
 'facing_poison': '1',
 'facing_psychic': '2',
 'facing_rock': '1',
 'facing_steel': '1',
 'facing_water': '0.5',
 'generation': '1',
 'height_m': '0.7',
 'hp': '45',
 'is_legendary': '0',
 'japanese_name': 'Fushigidaneã\x83\x95ã\x82·ã\x82®ã\x83\x80ã\x83\x8d',
 'name': 'Bulbasaur',
 'percentage_male': '88.1',
 'pokedex_number': '1',
 'sp_attack': '65',
 'sp_defense': '65',
 'speed': '45',
 'type1': 'grass',
 'type2': 'poison',
 'weight_kg': '6.9'}


 ## Part 2 - Format Data to a Readable Format

In [5]:
""" pprint does provide a fairly clean format as is, however, there is too much data
for general viewing. Let's pick out a select few by searching for them in the newly created data set.
Note - As an example, we don't need to print out all data rows, we'll show a few.
"""
# create list of data to show
select_data = "name type1 type2 hp speed attack defense".split()
count = 0

for data_dict in new_poke_rows:
    if count == 0 or count == 254 or count == 758:
        print("********* New Pokemon *********")

    for dkey, dval in data_dict.items():
        for s in select_data:
            if s == dkey:
                if count == 0 or count == 254 or count == 758:
                    print("{}: {}".format(dkey, dval))
    
    count += 1


********* New Pokemon *********
attack: 49
defense: 49
hp: 45
name: Bulbasaur
speed: 45
type1: grass
type2: poison
********* New Pokemon *********
attack: 60
defense: 40
hp: 45
name: Torchic
speed: 45
type1: fire
type2: 
********* New Pokemon *********
attack: 75
defense: 50
hp: 70
name: Stufful
speed: 50
type1: normal
type2: fighting


 ## Part 3 - Identify outliers and bad data

In [6]:
""" Given the data, there is expected to be a large variety of ranges in the stats of each
pokemone (e.g., hp can vary from low to high values). However, we identify outliers that 
we may or may not want to keep. is_legendary is column to check if a pokemon is legendary. 
You can argue either way to keep or discard it. For now, I'll just be finding which are legendary.
"""
legendary = 0
non_legendary = 0
# is_legendary = 1, yes
# is_legendary = 0, no
for data_dict in new_poke_rows:
    for dkey, dval in data_dict.items():
        if dkey == "is_legendary":
            if dval == "1":
                legendary += 1
            else:
                non_legendary += 1

print("There are {} legendary Pokemon".format(legendary))
print("There are {} non-legendary Pokemon".format(non_legendary))

# We can see ~10% of the Pokemon can be considered outliers.


There are 97 legendary Pokemon
There are 963 non-legendary Pokemon


 ## Part 4 - Find Duplicates

In [7]:
""" First we'll check to see if there are any duplicates based on the name col. Other cols will have duplicates and that is expected (e.g., type1 - there is a limited number of options).
"""
# 
unique = [] 
duplicate = 0

for data_dict in new_poke_rows:
    if data_dict["name"] in unique:
        duplicate += 1
    else:
        unique.append(data_dict["name"])

print("There are {} duplicates".format(duplicate))


There are 259 duplicates


In [8]:
# One way to handle this is to not include them in the new_poke_rows data, so lets 
# adjust the creation of these to exclude them from the beginning

new_poke_rows = []
unique = [] 

for data_dict in poke_rows:
    if data_dict["name"] in unique:
        pass

    else:
        new_row = {}
        for dkey, dval in data_dict.items():
            if dkey in new_headers.keys():
                new_row[new_headers[dkey]] = dval
            else:
                new_row[dkey] = dval
        new_poke_rows.append(new_row)

        unique.append(data_dict["name"])

# now we can re-run the previous loop to check
duplicate = 0
unique = []

for data_dict in new_poke_rows:
    if data_dict["name"] in unique:
        duplicate += 1
    else:
        unique.append(data_dict["name"])

print("There are {} duplicates".format(duplicate))


There are 0 duplicates


 ## Part 5 - Conduct Fuzzy Matching

In [9]:
from fuzzywuzzy import fuzz
import random


In [10]:
""" To mimic fuzzy matching, I'm going to use a subset of my data, specifically Type2 == "".
Then I'm going to fill the missing spots with variations of "fire" and test how they compare to "fire". I'm also going to exlcude duplicates.
"""
variations = ["FIRE", "fire type", "Fire"]

sub_poke_rows = []
unique = [] 

for data_dict in poke_rows:
    if data_dict["name"] in unique:
        pass
    elif data_dict["type2"] != "":
        pass 
    else:
        type2 = random.choice(variations)
        new_row = {}
        for dkey, dval in data_dict.items():
            if dkey in new_headers.keys():
                new_row[new_headers[dkey]] = dval
            elif dkey == "type2":
                new_row[dkey] = type2
            else:
                new_row[dkey] = dval
        sub_poke_rows.append(new_row)

pprint.pprint(sub_poke_rows[0])


{'abilities': "['Blaze', 'Solar Power']",
 'attack': '52',
 'base_egg_steps': '5120',
 'base_happiness': '70',
 'base_total': '309',
 'capture_rate': '45',
 'classfication': 'Lizard PokÃ©mon',
 'defense': '43',
 'experience_growth': '1059860',
 'facing_bug': '0.5',
 'facing_dark': '1',
 'facing_dragon': '1',
 'facing_electric': '1',
 'facing_fairy': '0.5',
 'facing_fight': '1',
 'facing_fire': '0.5',
 'facing_flying': '1',
 'facing_ghost': '1',
 'facing_grass': '0.5',
 'facing_ground': '2',
 'facing_ice': '0.5',
 'facing_normal': '1',
 'facing_poison': '1',
 'facing_psychic': '1',
 'facing_rock': '2',
 'facing_steel': '0.5',
 'facing_water': '2',
 'generation': '1',
 'height_m': '0.6',
 'hp': '39',
 'is_legendary': '0',
 'japanese_name': 'Hitokageã\x83\x92ã\x83\x88ã\x82«ã\x82²',
 'name': 'Charmander',
 'percentage_male': '88.1',
 'pokedex_number': '4',
 'sp_attack': '60',
 'sp_defense': '50',
 'speed': '65',
 'type1': 'fire',
 'type2': 'FIRE',
 'weight_kg': '8.5'}


In [11]:
# TODO - The above code sets up the fuzzy matching, additional code should be added to seach through the sub_poke_rows for all variations. I'm assuming I found them and continuing on the fuzzy matching


In [12]:
# Fuzzy matching
# TODO - A more detailed loop should be implemented to check more variations then the subset I'm looking at.

compare = "fire"

# ratio
for v in variations:
    ratio = fuzz.ratio(v, compare)
    print("The ratio for {} and {} is: {}".format(v, compare, ratio))


The ratio for FIRE and fire is: 0
The ratio for fire type and fire is: 62
The ratio for Fire and fire is: 75


In [13]:
# partial ratio
for v in variations:
    ratio = fuzz.partial_ratio(v, compare)
    print("The partial ratio for {} and {} is: {}".format(v, compare, ratio))


The partial ratio for FIRE and fire is: 0
The partial ratio for fire type and fire is: 100
The partial ratio for Fire and fire is: 75


In [14]:
# token sort ratio
for v in variations:
    ratio = fuzz.token_sort_ratio(v, compare)
    print("The token sort ratio for {} and {} is: {}".format(v, compare, ratio))


The token sort ratio for FIRE and fire is: 100
The token sort ratio for fire type and fire is: 62
The token sort ratio for Fire and fire is: 100


In [15]:
# token set ratio
for v in variations:
    ratio = fuzz.token_set_ratio(v, compare)
    print("The token set ratio for {} and {} is: {}".format(v, compare, ratio))


The token set ratio for FIRE and fire is: 100
The token set ratio for fire type and fire is: 100
The token set ratio for Fire and fire is: 100


 ## Summary

 See DSC540_Paulovici_Midterm_summary.docx